In [1]:
import pandas as pd
import numpy as np
import keras
#import liberaries
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split


In [2]:
# Part-A

In [3]:
df = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')


In [4]:
df.shape

(1030, 9)

In [5]:
df.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
# Check the null values in the dataset
null=df.isnull()
null

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
1025,False,False,False,False,False,False,False,False,False
1026,False,False,False,False,False,False,False,False,False
1027,False,False,False,False,False,False,False,False,False
1028,False,False,False,False,False,False,False,False,False


In [7]:
null.sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [8]:
# select and store the the column which we want to predict. 
target = df["Strength"]
target

0       79.99
1       61.89
2       40.27
3       41.05
4       44.30
        ...  
1025    44.28
1026    31.18
1027    23.70
1028    32.77
1029    32.40
Name: Strength, Length: 1030, dtype: float64

In [9]:
# to ignore the last column of the dataset.
predictors=df.iloc[:,:-1]

In [10]:
# Normalize the data.
pred_norm = (predictors - predictors.mean())/predictors.std()

In [11]:
pred_norm.head(10)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069
5,-0.145138,0.464818,-0.846733,2.174405,-1.038638,-0.526262,-1.291914,0.701883
6,0.945704,0.244603,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
7,0.945704,0.244603,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,-0.279597
8,-0.145138,0.464818,-0.846733,2.174405,-1.038638,-0.526262,-1.291914,-0.279597
9,1.854740,-0.856472,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,-0.279597


In [12]:
# To know the number of columns of predictors
num_col = pred_norm.shape[1]
num_col

8

In [13]:
model_norm_D = Sequential()
# set the hidden layer, activation function relu, and input shape.
model_norm_D.add(Dense(10, activation='relu', input_shape=(num_col,)))
model_norm_D.add(Dense(10, activation='relu'))
model_norm_D.add(Dense(10, activation='relu'))
model_norm_D.add(Dense(1))
# set the optimizer adam and error= mean square error.
model_norm_D.compile(optimizer='adam', loss='mean_squared_error')

In [14]:
# list to store the msqe values.
error4 = []
# loop for 50 iterationns.
for msqe in range(50):
    # spliting data in X,y train and test.
    X_train, X_test, y_train, y_test = train_test_split(pred_norm, target, test_size=0.3)
    # model fitting the train and test data fit to the model and start trainning and testing of data.
    result = model_norm_D.fit(X_train, y_train, epochs=50, verbose=0, validation_data=(X_test, y_test))
    # last value of the msqe.
    mean_squared_error = result.history['val_loss'][-1]
    # append the mean squared error of every iteration on list.
    error4.append(mean_squared_error)
    print('Time #{}: mean_squared_error {}'.format(msqe+1, mean_squared_error))

Time #1: mean_squared_error 134.9947967529297
Time #2: mean_squared_error 114.77449798583984
Time #3: mean_squared_error 102.84402465820312
Time #4: mean_squared_error 49.909908294677734
Time #5: mean_squared_error 45.414180755615234
Time #6: mean_squared_error 44.40625
Time #7: mean_squared_error 40.567222595214844
Time #8: mean_squared_error 38.03172302246094
Time #9: mean_squared_error 38.80820846557617
Time #10: mean_squared_error 33.91246795654297
Time #11: mean_squared_error 40.082130432128906
Time #12: mean_squared_error 35.66022872924805
Time #13: mean_squared_error 35.603145599365234
Time #14: mean_squared_error 32.94260025024414
Time #15: mean_squared_error 28.833879470825195
Time #16: mean_squared_error 27.158456802368164
Time #17: mean_squared_error 33.37812042236328
Time #18: mean_squared_error 28.751972198486328
Time #19: mean_squared_error 27.6314697265625
Time #20: mean_squared_error 27.200021743774414
Time #21: mean_squared_error 33.02910232543945
Time #22: mean_square

In [15]:
# Average mean of msqe
avg_msqe = np.mean(error4)
print("Average Mean squared error:",avg_msqe)

Average Mean squared error: 33.733858528137205


In [16]:
# standard deviation of msqe
avg_std = np.std(error4)
print("standard deviation:",avg_std)

standard deviation: 22.584071835499646
